In [2]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

#visualize the data
print(df.head())
print(df.info())

                                             context  \
0  Een draft van deze visienota  is opgemaakt en ...   
1  Een draft van deze visienota  is opgemaakt en ...   
2  Een draft van deze visienota  is opgemaakt en ...   
3  Daarna  zal Fluvius  de visienota indienen bij...   
4  Wanneer ze zich daarna  opnieuw willen  inzett...   

                                            question  statistical    theme  \
0  1.Kan de VREG de opgestelde visienota van Fluv...            0  Energie   
1     a)Wat zijn de krijtlijnen  van deze visienota?            0  Energie   
2  b)Welke acties zullen op basis van deze visien...            0  Energie   
3  2.Wat is de reactie van de VREG op deze visien...            0  Energie   
4  1.Hoeveel 50-54, 55-59  en 60-plussers  zijn s...            1     Werk   

     file_name  
0  1919076.txt  
1  1919076.txt  
2  1919076.txt  
3  1919076.txt  
4  1919282.txt  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39125 entries, 0 to 39124
Data columns (to

In [4]:
# Thema-cluster mapping
theme_merge_map = {
    # Bestuur en Beleid
    "Lokale overheden en Binnenlands bestuur": "Bestuur en Beleid",
    "Vlaamse administratie": "Bestuur en Beleid",
    "Staatshervorming en Verhoudingen binnen de Belgische federale staat": "Bestuur en Beleid",
    "Vlaamse Regering": "Bestuur en Beleid",

    # Mobiliteit en Infrastructuur
    "Mobiliteit en Verkeer": "Mobiliteit en Infrastructuur",
    "Openbare werken": "Mobiliteit en Infrastructuur",
    "Ruimtelijke ordening": "Mobiliteit en Infrastructuur",

    # Economie en Arbeid
    "Werk": "Economie en Arbeid",
    "Economie": "Economie en Arbeid",
    "Sociale economie": "Economie en Arbeid",
    "Internationaal ondernemen": "Economie en Arbeid",

    # Welzijn en Gezondheid
    "Welzijn en Gezin": "Welzijn en Gezondheid",
    "Gezondheid": "Welzijn en Gezondheid",
    "Armoedebeleid": "Welzijn en Gezondheid",

    # Cultuur en Communicatie
    "Cultuur": "Cultuur en Communicatie",
    "Media": "Cultuur en Communicatie",
    "Taalgebruik": "Cultuur en Communicatie",

    # Onderwijs en Samenleving
    "Onderwijs en Vorming": "Onderwijs en Samenleving",
    "Gelijke kansen": "Onderwijs en Samenleving",
    "Jeugdbeleid": "Onderwijs en Samenleving",
    "Integratie en Inburgering": "Onderwijs en Samenleving",

    # Milieu en Landbouw
    "Natuur en Milieu": "Milieu en Landbouw",
    "Landbouw": "Milieu en Landbouw",
    "Dierenwelzijn": "Milieu en Landbouw",

    # Internationaal Beleid
    "Buitenlands beleid": "Internationaal Beleid",
    "Europese instellingen": "Internationaal Beleid",
    "Ontwikkelingssamenwerking": "Internationaal Beleid",
    "Oekraïnecrisis": "Internationaal Beleid",

    # Overige (apart laten tenzij weinig samples)
    "Financiën": "Financiën",
    "Begroting": "Begroting",
    "Wetenschap en Innovatie": "Wetenschap en Innovatie",
    "Toerisme": "Toerisme",
    "Justitie en Handhaving": "Justitie en Handhaving",
    "Brussel en de Vlaamse Rand": "Brussel en de Vlaamse Rand",
    "Sport": "Sport",
    "Onroerend erfgoed": "Onroerend erfgoed",
}

# Nieuwe kolom aanmaken met samengevoegde thema's
df["theme_merged"] = df["theme"].map(theme_merge_map).fillna("Onbekend")



In [5]:
# Drop unnecessary columns
if "TXT_file_name" in df.columns:
    df = df.drop(columns=["TXT_file_name"])
    df = df.drop(columns=["statistical"])


# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)


def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Remove patterns like 'a)', 'b)', etc.
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    text = re.sub(r'•', '', text)  # Remove bullet symbol
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

#df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text)

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme_merged"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme_merged"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme_merged"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme_merged"].isin(valid_themes)]

# Remove rows where the 'question' column has 5 or fewer words
df = df[df['question'].apply(lambda x: len(str(x).split()) > 7)]



C:\Users\corne\AppData\Local\Temp\ipykernel_26824\309580979.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


In [6]:
print(len(df))

29885


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Prepare features and labels
X = df["clean_text"]
y = df["theme_merged"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# TF-IDF + SVM pipeline
svm_model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=10000),            #maybe try binary=True also to see if it helps (Question Classification using Support Vector Machines wee sun lee)
    LinearSVC()
)

# Train
svm_model.fit(X_train, y_train)

# Predict + Evaluate
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))

                              precision    recall  f1-score   support

                   Begroting       0.85      0.62      0.72        56
           Bestuur en Beleid       0.74      0.67      0.70       413
  Brussel en de Vlaamse Rand       0.73      0.48      0.58        33
     Cultuur en Communicatie       0.67      0.48      0.56       177
          Economie en Arbeid       0.70      0.68      0.69       564
                   Financiën       0.85      0.67      0.75       101
       Internationaal Beleid       0.68      0.55      0.61       143
      Justitie en Handhaving       0.73      0.49      0.58        68
          Milieu en Landbouw       0.60      0.66      0.63       916
Mobiliteit en Infrastructuur       0.73      0.83      0.78      1590
                    Onbekend       0.64      0.58      0.61       462
    Onderwijs en Samenleving       0.61      0.64      0.62       694
           Onroerend erfgoed       0.84      0.41      0.55        63
                   